Tweaked the feature engineering specifically to optimize performance for the logistic regression model.

Future Work : Try out autoML techniques like ExhaustiveFeatureSelector

## LIBRARIES

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from scipy.special import softmax

from sklearn.metrics import log_loss, brier_score_loss
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report

## LOAD DATA

In [3]:
# Load data
train_data = pd.read_csv('/content/drive/MyDrive/HORSE DATA/trainData.csv')
test_data = pd.read_csv('/content/drive/MyDrive/HORSE DATA/testData.csv')

In [4]:
train_data

,Race_Time,Race_ID,Course,Distance,distanceYards,Prize,Going,Horse,Trainer,Jockey,...,Speed_2ndPreviousRun,NMFPLTO,MarketOdds_PreviousRun,MarketOdds_2ndPreviousRun,TrainerRating,JockeyRating,daysSinceLastRun,SireRating,DamsireRating,meanRunners
0,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Intervention,Michael Appleby,Aiden Brookes,...,70.0,0.875000,11.03,3.60,2.377268,2.925027,7.0,2.933961,0.467149,10.25
1,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Evocative Spark,Darryll Holland,Christian Howarth,...,48.0,0.181818,42.67,4.19,2.401274,2.611219,13.0,1.934009,0.459547,10.25
2,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Sluzewiec,Scott Dixon,Kieran O'Neill,...,59.0,0.000000,141.13,86.83,2.824967,2.925073,50.0,2.411403,0.456616,8.00
3,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Muscika,David O'Meara,Mark Winn,...,73.0,0.333333,8.97,12.86,2.317504,2.534689,38.0,2.639010,0.462397,10.00
4,02/01/2024 19:00:00,1935,Wolverhampton,6f 20y,1340,4972,Standard,Venturous,David Barron,David Probert,...,62.0,0.090909,44.84,18.20,2.292027,2.448742,24.0,2.494198,0.450770,11.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52094,31/12/2024 15:22:00,56178,Lingfield,1m 4f,2640,4711,Standard,Fullforward,Michael Madgwick,William Carson,...,43.0,0.833333,9.04,13.23,2.968319,2.767347,57.0,2.718093,0.480990,8.75
52095,31/12/2024 15:22:00,56178,Lingfield,1m 4f,2640,4711,Standard,Pablo Prince,Karen Jewell,Luke Morris,...,55.0,0.500000,48.38,5.09,3.233067,2.457842,40.0,3.019927,0.449697,11.50
52096,31/12/2024 15:22:00,56178,Lingfield,1m 4f,2640,4711,Standard,Paradoxical,Jennie Candlish,George Wood,...,58.0,0.800000,6.03,7.12,2.554826,2.555626,5.0,2.584142,0.450076,10.50
52097,31/12/2024 15:22:00,56178,Lingfield,1m 4f,2640,4711,Standard,Keen Interest,Alice Haynes,Kieran O'Neill,...,51.0,0.400000,26.57,22.16,2.443137,2.917672,15.0,2.632082,0.456602,10.75


## INPUT AND OUTPUT

In [5]:
# ADDED INFORMATION TO THE MODEL

# Relative speed
train_data['SpeedRel1'] = train_data.groupby('Race_ID')['Speed_PreviousRun'].transform(lambda x: (x - x.mean()) / x.std())

train_data['SpeedRel2'] = train_data.groupby('Race_ID')['Speed_2ndPreviousRun'].transform(lambda x: (x - x.mean()) / x.std())

# Rank of MarketOdds within race (lower = more favored)
train_data['OddsRank1'] = train_data.groupby('Race_ID')['MarketOdds_PreviousRun'].rank(method='min', ascending=True)

train_data['OddsRank2'] = train_data.groupby('Race_ID')['MarketOdds_2ndPreviousRun'].rank(method='min', ascending=True)

# Z-score of TrainerRating within each race
train_data['TrainerRating_rel'] = train_data.groupby('Race_ID')['TrainerRating'].transform(lambda x: (x - x.mean()) / x.std())

# Relative Damsire rating
train_data['DamsireRel'] = train_data.groupby('Race_ID')['DamsireRating'].transform(lambda x: (x - x.mean()) / x.std())

# MIGHT REQUIRE MORE TRANSFORMATION TO ADD INFORMATION

# Z-score of JockeyRating within each race
train_data['JockeyRating_rel'] = train_data.groupby('Race_ID')['JockeyRating'].transform(lambda x: (x - x.mean()) / x.std())

train_data['SireRel'] = train_data.groupby('Race_ID')['SireRating'].transform(lambda x: (x - x.mean()) / x.std())


In [6]:
INP = [
    # Historical form / fitness
    'SpeedRel1',
    'SpeedRel2',
    'OddsRank1',
    'OddsRank2',
    'daysSinceLastRun',

    # Ratings (aggregated skill indicators)
    'TrainerRating_rel',
    'JockeyRating',
    'SireRating',
    'DamsireRel',

    # Might Not use
    'SireRel',
    'JockeyRating_rel',
    'TrainerRating',
    'DamsireRating',

    # Demographics
    'Age',

    # Race configuration (distance = performance factor)
    'distanceYards',
    'Going'
]

In [7]:
# Add Binary Target: Top50%
train_data['Top50%'] = (train_data['Position'] <= (train_data['Runners'] // 2)).astype(int)

In [8]:
# Add Inverse Position + Empirical Probability

# Avoid division by zero — handle only valid positions
train_data = train_data[train_data['Position'] > 0].copy()

train_data['inv_pos'] = 1 / train_data['Position']
train_data['empirical_prob'] = train_data.groupby('Race_ID')['inv_pos'].transform(lambda x: x / x.sum())

In [9]:
# Confirm that probabilities sum to 1 per race
train_data.groupby('Race_ID')['empirical_prob'].sum().describe()

,empirical_prob
count,5.337000e+03
mean,1.000000e+00
std,1.057584e-16
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [10]:
train_data['Win'] = (train_data['Position'] == 1).astype(int)

## CLEAN DATA


Remove NAN data points

In [11]:
# Fill NA values

# Show count of NaNs per column
train_data.isna().sum()

,0
Race_Time,0
Race_ID,0
Course,0
Distance,0
distanceYards,0
Prize,0
Going,0
Horse,0
Trainer,0
Jockey,0


In [12]:
cols_with_na = train_data.columns[train_data.isna().sum() > 0].tolist()
cols_with_na

['Speed_PreviousRun',
 'Speed_2ndPreviousRun',
 'NMFPLTO',
 'MarketOdds_PreviousRun',
 'MarketOdds_2ndPreviousRun',
 'TrainerRating',
 'JockeyRating',
 'daysSinceLastRun',
 'SireRating',
 'DamsireRating',
 'meanRunners',
 'SpeedRel1',
 'SpeedRel2',
 'OddsRank1',
 'OddsRank2',
 'TrainerRating_rel',
 'DamsireRel',
 'JockeyRating_rel',
 'SireRel']

In [13]:
# First, group by Horse and get medians for each feature
for col in cols_with_na:
    train_data[col] = train_data.groupby('Horse')[col].transform(lambda x: x.fillna(x.median()))

In [14]:
# Still There are NaNs so remove those races
# First, flag races with any NaNs in selected inputs
incomplete_races = train_data[train_data[cols_with_na].isna().any(axis=1)]['Race_ID'].unique()
len(incomplete_races)

81

In [15]:
# Drop those races entirely
train_data = train_data[~train_data['Race_ID'].isin(incomplete_races)].reset_index(drop=True)

In [16]:
# Still incomplete race info?
incomplete_races = train_data[train_data[cols_with_na].isna().any(axis=1)]['Race_ID'].unique()
len(incomplete_races)

0

## PREDICT TOP50% AS OUTPUT

### SPLIT X and Y

In [17]:
# STEP 1: Get unique Race_IDs
unique_races = train_data['Race_ID'].unique()

In [18]:
# STEP 2: Stratify by win outcomes
train_races, val_races = train_test_split(
    unique_races,
    test_size=0.3,
    random_state=42
)

In [19]:
# STEP 3: Filter data based on race split
train_mask = train_data['Race_ID'].isin(train_races)
val_mask = train_data['Race_ID'].isin(val_races)

X_train = train_data.loc[train_mask, INP].copy()
y_train = train_data.loc[train_mask, 'Win'].values

X_val = train_data.loc[val_mask, INP].copy()
y_val = train_data.loc[val_mask, 'Win'].values

### SCALE FEATURES

In [20]:
going_rank = {
    'Firm': 1,
    'Good To Firm': 2,
    'Good': 3,
    'Good To Soft': 4,
    'Soft': 5,
    'Heavy': 6,
    'Standard': 7
}

In [21]:
# Identify numeric columns (all except 'Going')
numeric_features = [col for col in INP if col != 'Going']

In [22]:
# STEP 4: Preprocess
X_train['Going'] = X_train['Going'].map(going_rank)
X_val['Going'] = X_val['Going'].map(going_rank)

# Scale numeric features
scaler = StandardScaler()

X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_val[numeric_features] = scaler.transform(X_val[numeric_features])

In [23]:
len(X_val)

15220

### ADVANCED FEATURE ENGG

In [ ]:
# from mlxtend.feature_selection import ExhaustiveFeatureSelector
# from sklearn.linear_model import LogisticRegression

# efs = ExhaustiveFeatureSelector(LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
#                                 min_features=2,
#                                 max_features=16,
#                                 scoring='neg_log_loss',
#                                 cv=3)

# efs = efs.fit(X_train, y_train)
# best_features = list(efs.best_feature_names_)
# best_features

Features: 65519/65519

['SpeedRel1',
 'SpeedRel2',
 'OddsRank1',
 'OddsRank2',
 'daysSinceLastRun',
 'TrainerRating_rel',
 'JockeyRating',
 'SireRating',
 'JockeyRating_rel',
 'TrainerRating',
 'Age',
 'distanceYards']

### INPUT : X

In [ ]:
X_train.isna().sum()

### MODELS

In [24]:
# Add Race_ID back to validation set
# Assuming val_data is the DataFrame corresponding to X_val
val_data = X_val.copy()
val_data['Race_ID'] = train_data.loc[val_mask, 'Race_ID'].values
val_data['Position'] = train_data.loc[val_mask, 'Position'].values
val_data['TrueWin'] = y_val

In [25]:
def softmax_probs(x):
    e_x = np.exp(x - np.max(x))  # numerical stability
    return e_x / e_x.sum()

In [32]:
model = LogisticRegression(class_weight='balanced',max_iter=1000, random_state=42)

model.fit(X_train, y_train)

# Raw prediction probabilities
raw_probs = model.predict_proba(X_val)[:, 1]

# Attach to validation set
val_data['raw_prob'] = raw_probs

# Apply softmax per race
val_data['softmax_prob'] = val_data.groupby('Race_ID')['raw_prob'].transform(lambda x: softmax_probs(x.values))

In [33]:
# Group by Race_ID and sum softmax_prob
race_softmax_sums = val_data.groupby('Race_ID')['softmax_prob'].sum().reset_index()

# Optional: Rename column for clarity
race_softmax_sums.columns = ['Race_ID', 'Sum_Softmax_Prob']

# View the result
race_softmax_sums['Sum_Softmax_Prob'].unique()

array([1., 1., 1., 1., 1.])

In [34]:
race_softmax_sums

,Race_ID,Sum_Softmax_Prob
0,166,1.0
1,227,1.0
2,230,1.0
3,232,1.0
4,233,1.0
...,...,...
1572,56098,1.0
1573,56173,1.0
1574,56174,1.0
1575,56175,1.0


In [35]:
# Predicted class from raw (not softmax)
y_pred = (val_data['raw_prob'] >= 0.5).astype(int)

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()

sensitivity = tp / (tp + fn) if (tp + fn) else 0  # Recall
specificity = tn / (tn + fp) if (tn + fp) else 0

# Log Loss / Brier using softmax-normalized probs
true_labels = val_data['TrueWin'].values
softmax_probs_pred = val_data['softmax_prob'].values

logloss = log_loss(true_labels, softmax_probs_pred)
brier = brier_score_loss(true_labels, softmax_probs_pred)
bal_acc = balanced_accuracy_score(true_labels, y_pred)

In [36]:
from tabulate import tabulate

# Format as a pretty table
results = [
    ['Balanced Accuracy', f'{bal_acc*100:.4f}%'],
    ['Sensitivity (Recall)', f'{sensitivity*100:.4f}%'],
    ['Specificity', f'{specificity*100:.4f}%'],
    ['Log Loss (Softmax)', f'{logloss:.4f}'],
    ['Brier Score (Softmax)', f'{brier:.4f}']
]

print(tabulate(results, headers=['Metric', 'Value'], tablefmt='grid'))

+-----------------------+----------+
| Metric                | Value    |
+=======================+==========+
| Balanced Accuracy     | 62.0737% |
+-----------------------+----------+
| Sensitivity (Recall)  | 65.8011% |
+-----------------------+----------+
| Specificity           | 58.3462% |
+-----------------------+----------+
| Log Loss (Softmax)    | 0.3207   |
+-----------------------+----------+
| Brier Score (Softmax) | 0.0905   |
+-----------------------+----------+
